In [2]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import re
import thulac

In [3]:
#load all the xml files in the directory and decode them
paths = []
rootset = []
for path, subdirs, files in os.walk(r'C:\Users\romilda\Desktop\weibo scrapers\training'):
    for filename in files:
        f = os.path.join(path, filename)
        data = ET.parse(f).getroot()
        rootset.append(data)
#save the text data into dataframes
weibo_id = []
sen_id = []
content = []
polar = []
wb_len = []
for sets in rootset:
    for wb in sets:
        for stnc in wb:
            #print(str(stnc))
            weibo_id.append(re.findall("""{'id': '(.*)'""",str(wb.attrib))[0])
            sen_id.append(str(stnc.get('id')))
            content.append(str(stnc.text))
            polar.append(str(stnc.get('polarity')))
            wb_len.append(str(len(wb)))
dt = pd.DataFrame({'weibo_id':weibo_id,
                  'sen_id':sen_id,
                  'content':content,
                  'polar':polar,
                  'wb_length':wb_len})
dt.head()

,weibo_id,sen_id,content,polar,wb_length
0,4650,1,#菲军舰恶意撞击#又他妈一个猴子,NEG,1
1,4657,1,#菲军舰恶意撞击#搞死它死菲佣,NEG,1
2,4658,1,#菲军舰恶意撞击#哇靠！,None,3
3,4658,2,究竟还要忍多久！,None,3
4,4658,3,我们的军队是摆设吗？,NEG,3


In [5]:
#a function that combine sentences for each paragraph
def combine_sen(input_dt,i,sen_len,weibo_id,output_dt):
    j = output_dt['weibo_id'].count()+1 #to get the index of next record according to the size of existing output dataset
    temp = [weibo_id] #temp is the piece of record written to output dataset; write weibo_id into the record
    
    content = ''
    polar = 0
    for x in range(sen_len):
        content = content+' '+str(input_dt.iat[i,2]) #combine the sentences
        
        #get the overall polarity of the whole sentence
        if str(input_dt.iat[i,3])=='NEG':
            polar = polar-1
        elif str(input_dt.iat[i,3])=='POS':
            polar = polar+1
        
        i=i+1 #update the location of next piece of data
    
    temp.append(content)  #write the content into the record
    
    #write polarity of the paragraph into the record
    if polar>0:
        pl = 1 #here 1 means positive
    elif polar==0:
        pl = 0 #here 0 means neutral
    else:
        pl = -1 #here -1 means negative
    temp.append(pl)
    output_dt.loc[j] = temp

In [6]:
#combine the sentences
dts = pd.DataFrame(columns=['weibo_id','content','polar'])
total_rows = dt['weibo_id'].count()
i = 0
while i < total_rows: #
    sen_len = int(dt.iat[i,4])
    wb_id = dt.at[i,'weibo_id']
    combine_sen(dt,i,sen_len,wb_id,dts)
    i = i + sen_len
dts.head(3)

,weibo_id,content,polar
1,4650,#菲军舰恶意撞击#又他妈一个猴子,-1
2,4657,#菲军舰恶意撞击#搞死它死菲佣,-1
3,4658,#菲军舰恶意撞击#哇靠！ 究竟还要忍多久！ 我们的军队是摆设吗？,-1


In [8]:
#load stopwords which are mostly special characters, clean it for later use
st =  pd.read_excel(r'C:\Users\romilda\Desktop\weibo scrapers\\stopword.xlsx',header=None)
stopword = []
colname = st.columns[0]
for index,row in st.iterrows():
    stopword.append(row[colname].replace('\xa0',''))
    
#word segmentation
ud1 = r'C:\Users\romilda\Desktop\weibo scrapers\userdict.txt'#import userdict
thu1 = thulac.thulac(user_dict=ud1,seg_only=True)
for index,row in dts.iterrows():
    temp = thu1.cut(row['content'])
    words = '' 
    for item in temp:
        if item[0] not in stopword:
            words = words+' '+item[0]
        dts.loc[index,'content'] = words[1:]
dts.head(3)

Model loaded succeed


,weibo_id,content,polar
1,4650,菲 军舰 恶意 撞击 又 他 妈 一个 猴子,-1
2,4657,菲 军舰 恶意 撞击 搞 死 它 死 菲 佣,-1
3,4658,菲 军舰 恶意 撞击 哇 靠 究竟 还要 忍 多久 我们 的 军队 是 摆设 吗,-1


In [10]:
ot = dts[['content','polar']] #save only columns of content and polarity
ot.to_csv('C:\\Users\\romilda\\Desktop\\weibo scrapers\\seg_train.csv',encoding='utf-8-sig',index=False)

In [1]:
#need to restart the kernel everytime building the model
import pandas as pd
import numpy as np
import random
import keras
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pickle

#prepare the train set and test set
inpt = pd.read_csv(r'C:\\Users\\romilda\\Desktop\\weibo scrapers\\seg_train.csv',header='infer')
pos = inpt[inpt['polar'] == 1]
neur = inpt[inpt['polar'] == 0]
neg= inpt[inpt['polar'] == -1].sample(n=350)
rd = pd.concat([pos,neur,neg],axis=0)
rd['content']= rd['content'].apply(lambda x: x.lower()) #convert capital letters into lower case
print(int(rd[rd['polar'] == 1].size/2)) #print the number of rows with positive label
print(int(rd[rd['polar'] == 0].size/2)) #print the number of rows with neural label
print(int(rd[rd['polar'] == -1].size/2)) #print the number of rows with negative label
tk = keras.preprocessing.text.Tokenizer(num_words=None, lower=True, split=' ', char_level=False, oov_token=True)
tk.fit_on_texts(rd['content'].values)
X = tk.texts_to_sequences(rd['content'].values)
X = pad_sequences(X)
#save the dictionary of tokenizer which will be loaded for prediction data
with open('C:\\Users\\romilda\\Desktop\\weibo scrapers\\tokenizer.pickle', 'wb') as handle:
        pickle.dump(tk, handle, protocol=pickle.HIGHEST_PROTOCOL)
Y = pd.get_dummies(rd['polar']).values #convert the labels into dummy variables that machine can understand
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42) #67% data is used as train data and 33% is used as test data

#build the model
max_features = 6500
embed_dim = 192 #128
lstm_out = 196
batch_size_ = 84 # the maximum length is 112
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

#train the model with training set
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size_, verbose = 2)

Using TensorFlow backend.


294
343
350
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 94, 192)           1248000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 94, 192)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               304976    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 591       
Total params: 1,553,567
Trainable params: 1,553,567
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
 - 4s - loss: 1.0966 - acc: 0.3631
Epoch 2/10
 - 2s - loss: 1.0799 - acc: 0.4750
Epoch 3/10
 - 2s - loss: 1.0196 - acc: 0.5416
Epoch 4/10
 - 2s - loss: 0.9525 - acc: 0.6324
Epoch 5/10
 - 3s - loss: 0.8290 - acc: 0.6989
Epoch 6/10
 - 3s 

In [2]:
# to calculate the accuracy of prediction on train data 
y=model.predict(X_test)
label=y.argmax(axis=1)
# 0: -1 1: 0  2: 1
sum(label==0)
sum(label==1)
sum(label==2)
Ytest_label=np.array(list(map(lambda x: list(x).index(1),Y_test)))
(Ytest_label==label).mean()

0.6411042944785276

In [3]:
# to save the architecture of the model, the weight, the training configuration (loss, optimizer) and the state of the optimizer
model.save('C:\\Users\\romilda\\Desktop\\weibo scrapers\\my_model_weights.h5')

In [4]:
# this section of code is not needed for the project, but may be used if need to load the model for further use
from keras.models import load_model
n_model = load_model('C:\\Users\\romilda\\Desktop\\weibo scrapers\\my_model_weights.h5')

In [17]:
tk = keras.preprocessing.text.Tokenizer(num_words=None, lower=True, split=' ', char_level=False, oov_token=True)
# load the dictionary of indexing from previous training process
with open('C:\\Users\\romilda\\Desktop\\weibo scrapers\\tokenizer.pickle', 'rb') as handle:
    tk = pickle.load(handle)
    
# function that converts segmented words into index with the same dictionary as data training
def convert_to_sq(lst):
    x = tk.texts_to_sequences(lst)
    x = pad_sequences(x, maxlen=94)
    return x

# prediction with comments as input
km = pd.read_excel(r'C:\\Users\\romilda\\Desktop\\weibo scrapers\\seg_comments_with_index.xlsx')
yypred = np.zeros((len(km),3))
for i in range(len(km)):
    pdc = convert_to_sq([str(km['content'][i])])
    yypred[i]=model.predict(pdc,batch_size=1)
label=yypred.argmax(axis=1)-1
pl = pd.DataFrame(label,columns=['pred_polarity'])
fc = pd.concat([km,pl],axis=1)
fc.to_excel('C:\\Users\\romilda\\Desktop\\weibo scrapers\\pred_comments.xlsx',encoding='utf-8-sig',index=False)